# Tutorial 2: Inference

In [1]:
import libspn as spn
import tensorflow as tf

def init_const(v):
    return tf.initializers.constant(v)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### Building a Test Graph with Initialized Weights

In [2]:
indicators_x = spn.IndicatorLeaf(num_vars=2, num_vals=2, name="indicators_x")
sum_11 = spn.Sum((indicators_x, [0,1]), name="sum_11")
sum_11.generate_weights(init_const([0.4, 0.6]))
sum_12 = spn.Sum((indicators_x, [0,1]), name="sum_12")
sum_12.generate_weights(init_const([0.1, 0.9]))
sum_21 = spn.Sum((indicators_x, [2,3]), name="sum_21")
sum_21.generate_weights(init_const([0.7, 0.3]))
sum_22 = spn.Sum((indicators_x, [2,3]), name="sum_22")
sum_22.generate_weights(init_const([0.8, 0.2]))
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
root.generate_weights(init_const([0.5, 0.2, 0.3]))
latent_indicator_y = root.generate_latent_indicators(name="latent_indicator_y")

Instructions for updating:
Colocations handled automatically by placer.


[WARNING] [tensorflow:new_func] From /home/jos/.local/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


### Visualizing the SPN Graph

In [ ]:
spn.display_spn_graph(root)

### Add Value Ops

In [3]:
init_weights = spn.initialize_weights(root)
marginal_val = root.get_value(inference_type=spn.InferenceType.MARGINAL)
mpe_val = root.get_value(inference_type=spn.InferenceType.MPE)

### Calculate Values

In [4]:
indicators_x_arr = [[0, 1],
           [1, 0],
           [1,-1],
           [-1,-1]]

latent_indicator_y_arr = [[-1]] * 4

with tf.Session() as sess:
    sess.run(init_weights)
    marginal_val_arr = sess.run(marginal_val, feed_dict={indicators_x: indicators_x_arr, latent_indicator_y: latent_indicator_y_arr})
    mpe_val_arr = sess.run(mpe_val, feed_dict={indicators_x: indicators_x_arr, latent_indicator_y: latent_indicator_y_arr})

print(marginal_val_arr)
print(mpe_val_arr)

[[0.08199999]
 [0.52199996]
 [0.69      ]
 [1.        ]]
[[0.06 ]
 [0.216]
 [0.216]
 [0.216]]


### Add MPE State Ops

In [5]:
mpe_state_gen = spn.MPEState(value_inference_type=spn.InferenceType.MPE)
indicators_x_state, latent_indicator_y_state = mpe_state_gen.get_state(root, indicators_x, latent_indicator_y)

Instructions for updating:
Use tf.random.categorical instead.


[WARNING] [tensorflow:new_func] From /home/jos/.local/lib/python3.5/site-packages/tensorflow_probability/python/distributions/categorical.py:263: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.random.categorical instead.


In [6]:
with spn.session() as (sess, _):
    init_weights.run()
    indicators_x_state_arr, latent_indicator_y_state_arr = sess.run([indicators_x_state, latent_indicator_y_state], 
                                              feed_dict={indicators_x: [[-1,-1]], 
                                                         latent_indicator_y: [[-1]]})
    
print(indicators_x_state_arr)
print(latent_indicator_y_state_arr)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


[WARNING] [tensorflow:new_func] From /home/jos/spn/libspn/libspn/session.py:34: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


[WARNING] [tensorflow:start_queue_runners] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


[[1 0]]
[[2]]
